In [30]:
import sys
import datetime
import pandas as pd
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium import webdriver

In [44]:
#Not implemented
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

In [32]:
driver = webdriver.Edge(service = Service(EdgeChromiumDriverManager().install()))

In [33]:
competitions = 'https://www.worldcubeassociation.org/competitions?event_ids%5B%5D=333&event_ids%5B%5D=222&event_ids%5B%5D=444&event_ids%5B%5D=555&event_ids%5B%5D=666&event_ids%5B%5D=777&region=all&search=&year=all+years&state=past&from_date=&to_date=&delegate=&display=list'

In [34]:
driver.get(competitions)

In [43]:
#Not implemented
cubes = ['222', '333', '444', '555', '666', '777']

In [36]:
competitions_we = driver.find_elements(By.CLASS_NAME, 'competition-link')

In [41]:
competition_catalog = {}
init = datetime.datetime.now()
for i, element in enumerate(competitions_we):
    u = 50/(e := len(competitions_we))
    competition_catalog[element.text] = {
        'url': element.find_element(By.TAG_NAME, 'a').get_attribute('href'),
        'country': element.find_element(By.TAG_NAME, 'span').get_attribute('class')}
    sys.stdout.write('\r')
    sys.stdout.write(
        "[{n}{s}] {p}% Elapsed time: {t}s".format(
            n = '='*round(i*u),
            s = ' '*round((e-i)*u),
            p = round((i/e)*100),
            t = (datetime.datetime.now() - init)))
    sys.stdout.flush()

[==================================================] 100% Elapsed time: 0:05:27.548066s

In [42]:
driver.quit()